In [15]:
import torch
from torch import nn
from torch.nn import functional as F

In [ ]:
class MedianPool2d(nn.Module):
    """ Median pool (usable as median filter when stride=1) module.
    
    Args:
         kernel_size: size of pooling kernel, int or 2-tuple
         stride: pool stride, int or 2-tuple
         padding: pool padding, int or 4-tuple (l, r, t, b) as in pytorch F.pad
         same: override padding and enforce same padding, boolean
    """
    def __init__(self, kernel_size=3, stride=1, padding=0, same=False):
        super(MedianPool2d, self).__init__()
        self.k = _pair(kernel_size)
        self.stride = _pair(stride)
        self.padding = _quadruple(padding)  # convert to l, r, t, b
        self.same = same

    def _padding(self, x):
        if self.same:
            ih, iw = x.size()[2:]
            if ih % self.stride[0] == 0:
                ph = max(self.k[0] - self.stride[0], 0)
            else:
                ph = max(self.k[0] - (ih % self.stride[0]), 0)
            if iw % self.stride[1] == 0:
                pw = max(self.k[1] - self.stride[1], 0)
            else:
                pw = max(self.k[1] - (iw % self.stride[1]), 0)
            pl = pw // 2
            pr = pw - pl
            pt = ph // 2
            pb = ph - pt
            padding = (pl, pr, pt, pb)
        else:
            padding = self.padding
        return padding
    
    def forward(self, x):
        # using existing pytorch functions and tensor ops so that we get autograd, 
        # would likely be more efficient to implement from scratch at C/Cuda level
        x = F.pad(x, self._padding(x), mode='reflect')
        x = x.unfold(2, self.k[0], self.stride[0]).unfold(3, self.k[1], self.stride[1])
        x = x.contiguous().view(x.size()[:4] + (-1,)).median(dim=-1)[0]
        return x

In [16]:
class DPP(nn.Module):
    """Squeeze and Excitation on means * variances"""

    def __init__(self, in_channels):
        super(DPP, self).__init__()
        self.beta = nn.Parameter(torch.Tensor([1.0]))
        self.alpha = nn.Parameter(torch.Tensor([1.0]))

    def forward(self, x):
        pass

In [17]:
x = torch.rand(2, 4, 112, 112)

In [18]:
def rho(x, beta=1.0, eps=1e-5):
    return torch.sqrt(x ** 2 + eps)

In [31]:
ker = torch.Tensor([[[[1/4, 1/4], [1/4, 1/4]]]]).repeat_interleave(4, dim=1)
F.conv2d(x, weight=ker, stride=2)

torch.Size([2, 1, 56, 56])

In [24]:
alpha + rho(x)

NameError: name 'alpha' is not defined

In [14]:
x.unfold(2, 2, 2).unfold(3, 2, 2)

torch.Size([2, 4, 56, 56, 2, 2])